# Alcholic Anonymous News articles extraction from BBC Newspaper

## Importing the necessary Libraries

In [1]:
from selenium.webdriver.support import expected_conditions as ec # WebDriverWait in combination with ExpectedCondition
from selenium.common.exceptions import TimeoutException # timeout exception
from selenium.webdriver.support.ui import WebDriverWait # Waiting provides mostly locating an element
from selenium.webdriver.chrome.options import Options # enables options in web browser
from selenium.webdriver.common.by import By # Locating Elements
from selenium import webdriver # web-based automation tool for Python
from newspaper import Article # Article scraping & curation
from bs4 import BeautifulSoup # Python library for pulling data out of HTML and XML files
from requests import get # standard for making HTTP requests in Python
import pandas as pd # library written for data manipulation and analysis
import sys, time #  System-specific parameters and functions

## Lists for Alcholic Anonymous News Articles extracted data

In [2]:
headlines, descriptions, dates, authors, news, keywords, summaries, urls, image_urls = [], [], [], [], [], [], [], [], []

## Scraping BBC Search results urls related to BBC Newspaper

## Finding the total no.of.pages from Google Search Results and passing the value through the while loop

In [3]:
%%time
options = Options()
options.headless = True
driver = webdriver.Chrome(options=options)
driver.get('https://www.bbc.co.uk/search?q=ALCOHOLICS+ANONYMOUS&filter=news')
wait = WebDriverWait(driver, 10)

index, maxclicks = 0, 25

while True:
    index += 1
    try:
        if index <= maxclicks:
            more_button = wait.until(ec.visibility_of_element_located((By.CLASS_NAME, "more"))).click()
            time.sleep(1)
    except TimeoutException:
        break
    sys.stdout.write('\r' + str(index) + ' : ' + str(maxclicks) + '\r')
    sys.stdout.flush()
    
news_elems = driver.find_elements_by_class_name('rs_touch')
for article in news_elems:
    urls.append(article.get_attribute('href'))
driver.quit()

Wall time: 44.5 s


## To remove duplicates urls entries in the list by executing below line

In [4]:
urls = list(dict.fromkeys(urls))
print(len(urls), type(urls))

243 <class 'list'>


## Scraping the Articles details from BBC Newspaper with the help of Newspaper

In [5]:
%%time
for index, url in enumerate(urls):
    try:
        # Parse the url to NewsPlease 
        article = Article(url)
        article.download()
        article.parse()

        try:
            # Extracts the Headlines 
            headlines.append(article.title)
        except:
            headlines.append(None)
            
        try:
            # Extracts the Descriptions
            descriptions.append(article.meta_description)
        except:
            descriptions.append(None)
            
        try:
            # Extracts the Authors
            authors.append(article.authors)
        except:
            authors.append(None)
            
        try:
            # Extracts the published dates
            dates.append(str(article.publish_date))
        except:
            dates.append(None)

        try:
            # Extracts the news articles
            news.append(article.text)
        except:
            news.append(None)
            
        try:
            # Extracts Keywords and Summaries
            article.nlp()
            keywords.append(article.keywords)
            summaries.append(article.summary)
        except:
            keywords.append(None)
            summaries.append(None)
                    
        try:
            # Extracts the image urls
            image_urls.append(article.top_image)
        except:
            image_urls.append(None)
            
    except:
        headlines.append(None)
        descriptions.append(None)
        authors.append(None)
        dates.append(None)
        news.append(None)
        keywords.append(None)
        summaries.append(None)
        image_urls.append(None)
        
    sys.stdout.write('\r' + str(index) + ' : ' + str(url) + '\r')
    sys.stdout.flush()

Wall time: 17min 15sc.co.uk/onthisday/hi/witness/january/1/newsid_4542000/4542042.stmml72265.stm621.stmstm


## Checking Array Length of each list to create DataFrame

In [6]:
print(len(headlines), len(descriptions), len(authors), len(dates), len(news), len(keywords), len(summaries), len(urls), len(image_urls))

243 243 243 243 243 243 243 243 243


## Creating a csv file after checking array length

In [7]:
tbl = pd.DataFrame({'Headlines' : headlines,
                    'Descriptions' : descriptions,
                    'Authors' : authors,
                    'Published_Dates' : dates, 
                    'Articles' : news,
                    'Keywords' : keywords,
                    'Summaries' : summaries,
                    'Source_URLs' : urls,
                    'Image_URLs' : image_urls})

tbl.to_csv('BBC.csv', index=False)